In [27]:
from langchain.prompts import ChatPromptTemplate
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [28]:
from PyPDF2 import PdfReader

# Ruta al archivo PDF
ruta_pdf = "../archivos/balonmano.pdf"

lector = PdfReader(ruta_pdf)

text = ""
for pagina in lector.pages:
    text += pagina.extract_text()

print(text)


 
 
 
 
  
EVOLUCIÓN DEL DEPORTE DE BALONMANO  
 
I. APROXIMACIÓN HISTÓRICA 
 
 
 
Ilustración 1 . Jugador de pelota. Cultura de Jalisco (Siglo III d. C), figura en barro 
de 13,2 cm 
 
 
 
Profesor, Juan de Dios Román Seco 
    2015 
 
 1 
  
 
  
 
EVOLUCION DEL DEPORT E DE BALONMANO                    
 
I. APROXIMACIÓN  HISTÓRICA 
 
ÍNDICE 
 
 A MODO DE INTRODUCCIÓN 
 
1. LOS ORÍGENES ANCESTRALES:  
Formas Primitivas de juegos con balón(800 a.c) 
2. JUEGOS PRECURSORES DE  LAS REGLAS 1870-1926) 
 Las primeras pautas 
Niño con varios padres 
3. INICIALES ESTR UCTURAS (1926-1946) 
   Hacia un deporte propio y organizado 
   Fundamentar Reglas como principio 
   La IAHF precursora de la IHF (1926/1945) 
4. LA FUNDACION DE LA  FEDERACION INTERNACIONAL 
   Nuevo pulso. (1946….) 
   Se inicia la Historia del Balonmano actual 
5. CRONOLOGÍA 
   De las formas primitivas al BM actual 
6. EL BALONMANO EN ESPAÑA 
   Las primeras noticias y diferentes vías 
   Otras fuentes 
   La fundación de 

In [29]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,  
    chunk_overlap=100, 
    length_function=len
)
chunks = text_splitter.split_text(text)


In [30]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


In [31]:
from langchain_chroma import Chroma

vector_store = Chroma.from_texts(
    texts=chunks,
    collection_name="balonmano",
    embedding=embeddings,
    persist_directory="./balonmano_datos",
)

In [32]:
results = vector_store.similarity_search_with_score(
    "Balonmano", k=3,
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content}")

* [SIM=1.074797] física y como ventajosos instrumentos para la transmisión de valores educativos. Así 
pues, no es baladí presuponer que el juego de  “Balón a la mano”, fu ese proclive como 
contenido de la educación fí sica en algunas de las escu elas españolas, como tantos 
otros juegos que se practicaban en aquella época”.  
 
Y continúa así… 
 
“Los Exploradores de España organizaron desde su fundación numerosos 
concursos y festivales gimnástico-deportivos. Entre sus deportes favoritos estaban el boxeo y el fútbol, aunque también se pusieron en práctica numerosos juegos del tratado del capitán Baden Powell  (1914), entre los que se encontraban algunos con balón. Por 
eso no debería parecer excepcional que dicha asociación también se iniciara, desde 
principios de los años veinte  al juego de “Balón a la mano” de Jentzer , aspecto que 
podría relacionarse con la introducción del juego en algunos de sus conocidos 
campeonatos deportivos. Como prueba aportamo s la noticia que en 1930

In [33]:
retriever = vector_store.as_retriever(
    search_type="similarity",  search_kwargs={"k": 3}
)

In [34]:
# Prompt
template = """Answer the question based only on the following context:
{context} in Spanish .

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

ollama_llm = "mistral"
model_local = ChatOllama(model=ollama_llm)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model_local
    | StrOutputParser()
)

In [35]:
# Pregunta 1
chain.invoke("Qué es el balonmano?")

' El contexto proporciona información sobre un juego conocido como "Balón a la mano" de Jentzer, y que en ciertas escuelas españolas se practicaba en la época. Sin embargo, no se hace explícito el nombre "balonmano" en este contexto específico. Para obtener más detalles sobre cómo se llama este juego o cuál es su significado, podría ser necesario consultar fuentes adicionales que proporcionen información más exhaustiva sobre este tema.'

In [36]:
# Pregunta 2
chain.invoke("Háblame del balonmano en España")

' En este contexto, se plantea una tesis que ubica el origen del balonmano femenino en España en Asturias. Se menciona a la jugadora asturiana Purina Zapico, quien a través de contactos con maestros precursores del balonmano Max Heiser y Max Schelenz, adquirió conocimientos del deporte en Alemania. También se indica que Purina practicó la especialidad tanto a "once" como a "siete" y colaboró en ella. No hay información sobre el crecimiento general del balonmano en España, pero se menciona que se trata de un deporte en expansión debido a varios factores. Además, se sugiere que la evolución del juego de balonmano se produce a partir de notas, de un ruido, de un momento, alrededor de determinados climas, ambientes o contextos en la realidad propia del hombre a lo largo de la historia.'

In [37]:
# Pregunta 3
chain.invoke("Explícame el origen del balonmano")

' El juego mencionado como "Balón a la mano" parece ser el antecesor directo del Balonmano. Sin embargo, no hay una referencia clara en el contexto sobre su origen exacto. Según el contexto proporcionado, se puede deducir que este juego se practicaba en algunas escuelas españolas a principios del siglo XX y que los Exploradores de España lo introdujeron en algunos de sus campeonatos deportivos desde principios de los años veinte. Además, se menciona que el juego fue desarrollado en un torneo universitario femenino en 1935, como el primer deporte universitario femenino. Por otro lado, también se menciona la influencia del tratado del capitán Baden-Powell (1914), lo que sugiere una posible relación con los juegos de esa época en Gran Bretaña o posiblemente de origen británico. Sin embargo, esta información no se confirma explícitamente en el texto proporcionado.'